# Task 2 Coding Decision Tree:

### Spliting the data:

In [87]:
import pandas as pd
import numpy as np
from treelib import Node, Tree
df = pd.read_csv("agaricus-lepiota.data", header = None)

### Adding columns to the dataframe and splitting it into training and testing sets
 We also remove the entries with missing values

In [88]:
from sklearn.model_selection import train_test_split

df.columns = ['class', 'cap_shape', 'cap_surface', 'cap_color', 'bruises', 'odor', 
               'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color', 
               'stalk_shape', 'stalk_root', 'stalk_surface_above_ring', 'stalk_surface_below_ring', 'stalk_color_above_ring', 'stalk_color_below_ring', 
               'veil_type', 'veil_color', 'ring_number', 'ring_type', 'spore_print_color', 'population', 'habitat']

df = df.drop(df[df.stalk_root  == "?"].index) # drop rows with missing values

training, testing = train_test_split(df, test_size=0.2, random_state=42)

### The methods used to generate the Decision Tree

In [89]:

def entropy(target_col):
        elements,counts = np.unique(target_col,return_counts = True)
        entropy = 0
        for i in range(len(elements) ):
             entropy +=(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts))
        return entropy

def information_Gain(data,split_feature, target_col="class"):
        total_entropy = entropy(data[target_col])
        vals,counts= np.unique(data[split_feature],return_counts=True)
        weighted_entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_feature]==vals[i]).dropna()[target_col]) for i in range(len(vals))])
        information_gain = total_entropy - weighted_entropy
        return information_gain

def build_tree(data,features, target_col="class",parent_node_value = None,depth=-1, stopping_depth=3, tree =None):
        
        if len(np.unique(data[target_col])) <= 1 or depth == stopping_depth :
            
            value = np.unique(data[target_col])[0]
            tree.create_node(value , identifier=len(tree.all_nodes())+1, parent=parent_node_value)
            return tree
           
        elif len(data) == 0:
            node_Value = np.unique(data[target_col])[np.argmax(np.unique(data[target_col],return_counts=True)[1])]

            tree.create_node(node_Value, node_Value, parent=parent_node_value)
            return tree
        
        elif len(features) ==0:
            
            return tree 
            
        else:
            

            information_Gain_List =[]

            for feature in features:
                    information_Gain_List.append(information_Gain(data,feature))
            
            best_feature_index = np.argmax(information_Gain_List)
            best_feature = features[best_feature_index]
            
          
            if tree is None:
                decision_tree = Tree()
                decision_tree.create_node(tag=best_feature, identifier=best_feature, parent=None)
            else:
                decision_tree = tree
                decision_tree.create_node(tag=best_feature, identifier=best_feature , parent=parent_node_value)
            
            features = features.drop(best_feature)
           
            depth += 1
            for value in np.unique(data[best_feature]):
                
                sub_data = data.where(data[best_feature] == value).dropna()
                identifier = str(best_feature) + "_" + str(value)
                decision_tree.create_node(tag=value, identifier=identifier, parent=best_feature)
                build_tree(sub_data,features,target_col,identifier, depth, stopping_depth, decision_tree)
                
          
            return(decision_tree) 

### The Resulting Decision Tree:

In [105]:
decision_Tree = build_tree(training, training.columns[1:], stopping_depth = np.inf)
decision_Tree.show()

odor
├── a
│   └── e
├── c
│   └── p
├── f
│   └── p
├── l
│   └── e
├── m
│   └── p
├── n
│   └── spore_print_color
│       ├── k
│       │   └── e
│       ├── n
│       │   └── e
│       ├── r
│       │   └── p
│       └── w
│           └── cap_color
│               ├── c
│               │   └── e
│               ├── g
│               │   └── e
│               ├── n
│               │   └── e
│               ├── p
│               │   └── e
│               ├── w
│               │   └── p
│               └── y
│                   └── p
└── p
    └── p



The above decision tre is generated when there is no stopping depth

#### The following trees are generated when the stopping depth is set to 1,2,3,4:

In [107]:
decision_Tree_depth_1 = build_tree(training, training.columns[1:], stopping_depth = 1)
decision_Tree_depth_2 = build_tree(training, training.columns[1:], stopping_depth = 2)
decision_Tree_depth_3 = build_tree(training, training.columns[1:], stopping_depth = 3)
decision_Tree_depth_4 = build_tree(training, training.columns[1:], stopping_depth = 4)

print("DecisionTree with depth 1:")
decision_Tree_depth_1.show()

print("DecisionTree with depth 2:")
decision_Tree_depth_2.show()

print("DecisionTree with depth 3:")
decision_Tree_depth_3.show()

print("DecisionTree with depth 4:")
decision_Tree_depth_4.show()


DecisionTree with depth 1:
odor
├── a
│   └── e
├── c
│   └── p
├── f
│   └── p
├── l
│   └── e
├── m
│   └── p
├── n
│   └── spore_print_color
│       ├── k
│       │   └── e
│       ├── n
│       │   └── e
│       ├── r
│       │   └── p
│       └── w
│           └── e
└── p
    └── p

DecisionTree with depth 2:
odor
├── a
│   └── e
├── c
│   └── p
├── f
│   └── p
├── l
│   └── e
├── m
│   └── p
├── n
│   └── spore_print_color
│       ├── k
│       │   └── e
│       ├── n
│       │   └── e
│       ├── r
│       │   └── p
│       └── w
│           └── cap_color
│               ├── c
│               │   └── e
│               ├── g
│               │   └── e
│               ├── n
│               │   └── e
│               ├── p
│               │   └── e
│               ├── w
│               │   └── p
│               └── y
│                   └── p
└── p
    └── p

DecisionTree with depth 3:
odor
├── a
│   └── e
├── c
│   └── p
├── f
│   └── p
├── l
│   └── e
├── m
│   └── p
├── n
│   └── 

In [91]:
def predict(df, tree):  
    
    node = tree.get_node(tree.root)
    while not node.is_leaf():
        
        feature = node.tag 
        value = df[feature]
        
        tree_feature = tree.get_node(feature + "_" + value)
        children = tree.children(tree_feature.identifier)
        if (children[0].is_leaf()):
             return children[0].tag
        else:
             
            node = children[0]
         

def accuracy(df, tree):
    correct = 0
    for i in range(len(df)):
        predicted = predict(df.iloc[i], tree)
        ground_truth = df.iloc[i]["class"]
        if predicted == ground_truth:
            correct += 1
       
    return np.divide(correct,len(df))

In [103]:


print(accuracy (testing, x))


1.0
